In [18]:
import os
import sys
import random
from ultralytics import YOLO
from IPython.display import Image

# Add the 'src' directory to the Python path
sys.path.append(os.path.abspath(os.path.join('..', 'src')))

# Now we can import your config file
from config import Config

In [30]:
# --- 1. Define Model Path (robust) ---
model_name = 'best.pt'
candidate_paths = [
    os.path.join('..', 'models', 'weights', model_name),                       # ../models/weights/best.pt
    os.path.join('..', Config.MODEL_PHASE2_WEIGHTS),                           # runs/detect/yolov8_phase2_fine/weights/best.pt
    Config.MODEL_PHASE2_WEIGHTS,                                               # relative runs/... (may fail depending on CWD)
    # Ensure REPO_ROOT is defined if you use this line in config.py
    # str(Config.REPO_ROOT / 'models' / 'weights' / model_name)                  # absolute path variant
]

# Find the first existing model path
model_path = next((p for p in candidate_paths if p and os.path.exists(p)), None)

print("Checked model paths:")
for p in candidate_paths:
    # Check existence relative to notebook or use absolute paths if defined
    check_path = p if os.path.isabs(p) else os.path.join(os.path.dirname(os.getcwd()), p)
    print("  ", p, "->", "OK" if os.path.exists(check_path) else "MISSING")


if not model_path:
    # If using relative path for Config.MODEL_PHASE2_WEIGHTS, adjust the error message or logic
    model_path_from_config = os.path.join('..', Config.MODEL_PHASE2_WEIGHTS)
    if os.path.exists(model_path_from_config):
        model_path = model_path_from_config
    else:
        raise FileNotFoundError(f"Model file not found. Checked: {candidate_paths}. Place 'best.pt' in '../models/weights/' or ensure '{model_path_from_config}' exists.")

print("Using model:", model_path)


# --- 2. Define Test Image Path ---

# --- Option A: Test using a random image from the dataset (Commented out) ---
# images_dir = Config.IMAGES_DIR
# if not os.path.isdir(images_dir):
#     raise FileNotFoundError(f"Images dir not found: {images_dir}")
# test_image_name = random.choice(os.listdir(images_dir))
# image_path = os.path.join(images_dir, test_image_name)
# print("Using image:", image_path)

# --- Option B: Test using your specific image ---
test_image_name = 'test/test_pacarku.jpeg'
# Assuming test_salma.jpeg is in the same 'notebooks' folder as this notebook
image_path = os.path.abspath(test_image_name) # Use absolute path
if not os.path.exists(image_path):
    raise FileNotFoundError(f"Your image not found at {image_path}. Make sure '{test_image_name}' is in the 'notebooks' folder.")
print("Using image:", image_path)

Checked model paths:
   ../models/weights/best.pt -> MISSING
   ../runs/detect/yolov8_phase2_fine/weights/best.pt -> MISSING
   runs/detect/yolov8_phase2_fine/weights/best.pt -> OK
Using model: ../models/weights/best.pt
Using image: /Users/macm4/repositories/Machine Learning Model/modisch-model/notebooks/test/test_pacarku.jpeg


In [31]:

# --- 3. Load model and run prediction with debug info ---
model = YOLO(model_path)

# Lower max_det and explicit conf to avoid NMS/timeouts dropping detections
results = model.predict(source=image_path, conf=0.2, iou=0.45, max_det=200, save=True)

res = results[0]
n_boxes = len(res.boxes)
print(f"Detections: {n_boxes}")
if n_boxes == 0:
    print("No boxes detected — try lowering conf or checking model weights.")
else:
    # print box coords, conf, class id/name
    for i, box in enumerate(res.boxes):
        xyxy = box.xyxy[0].cpu().numpy()
        conf = float(box.conf[0].cpu().numpy()) if hasattr(box, 'conf') else None
        cls_id = int(box.cls[0].cpu().numpy()) if hasattr(box, 'cls') else None
        cls_name = model.names[cls_id] if cls_id is not None and cls_id in model.names else str(cls_id)
        print(f" {i}: bbox={xyxy.tolist()}, conf={conf:.3f}, class={cls_id} ({cls_name})")

# Show annotated image produced by YOLO
annotated_dir = res.save_dir  # directory where YOLO saved annotated image
annotated_path = os.path.join(annotated_dir, test_image_name)
print("Annotated image saved to:", annotated_path)

from IPython.display import Image, display
display(Image(filename=annotated_path))


image 1/1 /Users/macm4/repositories/Machine Learning Model/modisch-model/notebooks/test/test_pacarku.jpeg: 512x288 1 jacket, 1 pants, 24.1ms
Speed: 1.7ms preprocess, 24.1ms inference, 0.6ms postprocess per image at shape (1, 3, 512, 288)
Results saved to /Users/macm4/repositories/Machine Learning Model/modisch-model/runs/detect/predict7
Detections: 2
 0: bbox=[0.0, 584.9345092773438, 495.9247131347656, 1273.9256591796875], conf=0.491, class=0 (jacket)
 1: bbox=[374.59320068359375, 1178.33251953125, 533.6094360351562, 1279.553955078125], conf=0.423, class=2 (pants)
Annotated image saved to: /Users/macm4/repositories/Machine Learning Model/modisch-model/runs/detect/predict7/test/test_pacarku.jpeg


FileNotFoundError: [Errno 2] No such file or directory: '/Users/macm4/repositories/Machine Learning Model/modisch-model/runs/detect/predict7/test/test_pacarku.jpeg'